In [1]:
!pip install multimodal-transformers
# from imblearn.over_sampling import SMOTE, ADASYN
# !pip install imblearn
# from imblearn.over_sampling import SMOTENC

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from dataclasses import dataclass, field
import json
import logging
import os
from typing import Optional

import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoConfig,
    Trainer,
    EvalPrediction,
    set_seed
)
from transformers.training_args import TrainingArguments

from multimodal_transformers.data import load_data_from_folder
from multimodal_transformers.model import TabularConfig
from multimodal_transformers.model import AutoModelWithTabular

logging.basicConfig(level=logging.INFO)
os.environ['COMET_MODE'] = 'DISABLED'

/home/ayushmt119/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch

if torch.cuda.is_available():    
    print("GPU Available, using GPU \n")
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

GPU Available, using GPU 



In [4]:
import pandas as pd

dataset_dir_path = '/home/ayushmt119/content/codes/hs_cleaned_rank4_scale3_gen2.csv'
columns = ["HateSpeech", "BestIntent", "Target"]
full_df = pd.read_csv(dataset_dir_path)
data_df = pd.read_csv(dataset_dir_path , usecols = columns)
# data_df.dropna(inplace=True)
# train_df['Aggressive'] = train_df['Aggressive'].apply(lambda x: x-1)
# data_df['Relevance'] = data_df['Relevance'].apply(lambda x: x-1)
data_df.dropna(inplace=True)
data_df['BestIntent'] = data_df['BestIntent'].apply(lambda x: 0.0 if x == 'Informative' else (1.0 if x == 'Questioning' else (2.0 if x == 'Positive' else 3.0)))
data_df['Length'] = data_df['HateSpeech'].apply(lambda x: float(len(x)))
# ['Informative', 'Questioning','Positive','Denouncing']
data_df.head(100)

,HateSpeech,BestIntent,Target,Length
0,"Women must not have equal rights, they are mer...",0.0,WOMEN,109.0
1,Children with Down Syndrome should be isolated...,0.0,DISABLED,103.0
2,You are a Muslim and I hate everything about you.,0.0,MUSLIMS,49.0
3,Allowing gays to donate blood compromises pati...,2.0,LGBT+,90.0
4,"A woman is an emotional child, she is incapabl...",0.0,WOMEN,137.0
...,...,...,...,...
95,Children shouldn't be raised by lesbians! They...,1.0,LGBT+,77.0
96,Gay men are basically violent criminals.,1.0,LGBT+,40.0
97,Immigrants rape our women and they are not wel...,1.0,MIGRANTS,56.0
98,"If Islam really was a peaceful religion, we’d ...",0.0,MUSLIMS,115.0


In [5]:
data_df.count

<bound method DataFrame.count of                                              HateSpeech  BestIntent    Target  \
0     Women must not have equal rights, they are mer...         0.0     WOMEN   
1     Children with Down Syndrome should be isolated...         0.0  DISABLED   
2     You are a Muslim and I hate everything about you.         0.0   MUSLIMS   
3     Allowing gays to donate blood compromises pati...         2.0     LGBT+   
4     A woman is an emotional child, she is incapabl...         0.0     WOMEN   
...                                                 ...         ...       ...   
3478  This came straight from an imam: they believe ...         0.0   MUSLIMS   
3479    Asians are genetically inferior to other races.         0.0     OTHER   
3480  The West has been attacked by immigrants. They...         0.0  MIGRANTS   
3481                Gay people are a burden to society.         1.0     LGBT+   
3482  Migrants are here to use our money. It is a my...         1.0  MIGRANT

In [6]:
data_df['BestIntent'].value_counts()

0.0    1800
1.0     741
2.0     571
3.0     371
Name: BestIntent, dtype: int64

In [7]:
data_df.dropna()

,HateSpeech,BestIntent,Target,Length
0,"Women must not have equal rights, they are mer...",0.0,WOMEN,109.0
1,Children with Down Syndrome should be isolated...,0.0,DISABLED,103.0
2,You are a Muslim and I hate everything about you.,0.0,MUSLIMS,49.0
3,Allowing gays to donate blood compromises pati...,2.0,LGBT+,90.0
4,"A woman is an emotional child, she is incapabl...",0.0,WOMEN,137.0
...,...,...,...,...
3478,This came straight from an imam: they believe ...,0.0,MUSLIMS,234.0
3479,Asians are genetically inferior to other races.,0.0,OTHER,47.0
3480,The West has been attacked by immigrants. They...,0.0,MIGRANTS,63.0
3481,Gay people are a burden to society.,1.0,LGBT+,35.0


In [8]:
data_df['BestIntent'].value_counts()

0.0    1800
1.0     741
2.0     571
3.0     371
Name: BestIntent, dtype: int64

In [9]:
inf = []
qs = []
pos = []
den = []
for hs in range(len(data_df)):
  i = data_df.iloc[hs][1]
  if i==0.0:
    inf.append(data_df.iloc[hs][0])
  if i==1.0:
    qs.append(data_df.iloc[hs][0])
  if i==2.0:
    pos.append(data_df.iloc[hs][0])
  if i==3.0:
    den.append(data_df.iloc[hs][0])

In [10]:
data_df['Target'].value_counts()

MUSLIMS     942
MIGRANTS    619
WOMEN       510
LGBT+       449
JEWS        396
POC         292
DISABLED    161
OTHER       114
Name: Target, dtype: int64

In [11]:
td = {'MUSLIMS':0,'MIGRANTS':1,'WOMEN':2,'LGBT+':3,'JEWS':4,'POC':5,'DISABLED':6,'OTHER':7}

In [12]:
data_df['Target'] = data_df['Target'].apply(lambda x: int(td[x]))

In [13]:
data_df.head(10)

,HateSpeech,BestIntent,Target,Length
0,"Women must not have equal rights, they are mer...",0.0,2,109.0
1,Children with Down Syndrome should be isolated...,0.0,6,103.0
2,You are a Muslim and I hate everything about you.,0.0,0,49.0
3,Allowing gays to donate blood compromises pati...,2.0,3,90.0
4,"A woman is an emotional child, she is incapabl...",0.0,2,137.0
5,Jews control the u.s. government.,0.0,4,33.0
6,"Allahu Akbar, this means Death to all! And if ...",0.0,0,114.0
7,Immigrants are all illegal.,1.0,1,27.0
8,Foreigners are a burden to our country.,2.0,1,39.0
9,"I'm proud to be white, blacks are a disgrace t...",3.0,5,57.0


In [14]:
data_df.isnull().sum().sum()

0

In [15]:
data_df.groupby(['BestIntent']).describe()

Target                                               Length  \
             count      mean       std  min  25%  50%  75%  max   count   
BestIntent                                                                
0.0         1800.0  2.150000  2.006590  0.0  0.0  2.0  4.0  7.0  1800.0   
1.0          741.0  2.164642  1.978914  0.0  0.0  2.0  4.0  7.0   741.0   
2.0          571.0  2.464098  2.052817  0.0  1.0  2.0  4.0  7.0   571.0   
3.0          371.0  2.460916  2.123347  0.0  0.0  2.0  4.0  7.0   371.0   

                                                                 
                 mean        std   min   25%   50%   75%    max  
BestIntent                                                       
0.0         78.121111  39.933544  11.0  49.0  71.0  97.0  371.0  
1.0         76.052632  41.057622  18.0  47.0  68.0  95.0  336.0  
2.0         76.992995  42.027643  12.0  47.0  68.0  98.0  346.0  
3.0         75.221024  41.507338  16.0  45.0  68.0  94.5  379.0

In [16]:
# train_df['Aggressive'].value_counts()
data_df['BestIntent'].value_counts()

0.0    1800
1.0     741
2.0     571
3.0     371
Name: BestIntent, dtype: int64

In [17]:
# counterspeech = data_df['CounterSpeech'].values
# hatespeech = data_df['HateSpeech'].values
# rel_labels = data_df['Relevance'].values
# print(len(rel_labels), len(counterspeech), len(hatespeech))

In [18]:
# example of random oversampling to balance the class distribution
from collections import Counter
from sklearn.datasets import make_classification
# from imblearn.over_sampling import RandomOverSampler
# !pip install pandas
import pandas as pd
# oversample = RandomOverSampler(sampling_strategy='minority')
train_df, val_df, test_df = np.split(data_df.sample(frac=1), [int(.8*len(data_df)), int(.9 * len(data_df))])
print('Num examples train-val-test')
# smote_nc = SMOTENC(categorical_features=[0], random_state=0)
# X, y = oversample.fit_resample(train_df['HateSpeech'].to_numpy().reshape(-1, 1), train_df['BestIntent'])
# print(sorted(Counter(y).items()))
# X, y = oversample.fit_resample(X, y)
# print(sorted(Counter(y).items()))
# X, y = oversample.fit_resample(X, y)
# print(X, X.shape)
# print(sorted(Counter(y).items()))

# from google.colab import files

train_df.to_csv('train.csv')
val_df.to_csv('val.csv')
test_df.to_csv('test.csv')
# xy = {'HateSpeech':X.reshape(1,-1)[0], 'BestIntent': y}

# files.download('train.csv')
# files.download('val.csv')
# files.download('test.csv')

Num examples train-val-test


In [19]:
data_df['Target'].value_counts()

0    942
1    619
2    510
3    449
4    396
5    292
6    161
7    114
Name: Target, dtype: int64

In [20]:
# train_df = pd.DataFrame(xy)
# from google.colab import files
# print(len(train_df), len(val_df), len(test_df))

In [21]:
@dataclass
class ModelArguments:
  """
  Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
  """

  model_name_or_path: str = field(
      metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
  )
  config_name: Optional[str] = field(
      default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
  )
  tokenizer_name: Optional[str] = field(
      default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
  )
  cache_dir: Optional[str] = field(
      default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
  )


@dataclass
class MultimodalDataTrainingArguments:
  """
  Arguments pertaining to how we combine tabular features
  Using `HfArgumentParser` we can turn this class
  into argparse arguments to be able to specify them on
  the command line.
  """

  data_path: str = field(metadata={
                            'help': 'the path to the csv file containing the dataset'
                        })
  column_info_path: str = field(
      default=None,
      metadata={
          'help': 'the path to the json file detailing which columns are text, categorical, numerical, and the label'
  })

  column_info: dict = field(
      default=None,
      metadata={
          'help': 'a dict referencing the text, categorical, numerical, and label columns'
                  'its keys are text_cols, num_cols, cat_cols, and label_col'
  })

  categorical_encode_type: str = field(default='ohe',
                                        metadata={
                                            'help': 'sklearn encoder to use for categorical data',
                                            'choices': ['ohe', 'binary', 'label', 'none']
                                        })
  numerical_transformer_method: str = field(default='yeo_johnson',
                                            metadata={
                                                'help': 'sklearn numerical transformer to preprocess numerical data',
                                                'choices': ['yeo_johnson', 'box_cox', 'quantile_normal', 'none']
                                            })
  task: str = field(default="classification",
                    metadata={
                        "help": "The downstream training task",
                        "choices": ["classification", "regression"]
                    })

  mlp_division: int = field(default=4,
                            metadata={
                                'help': 'the ratio of the number of '
                                        'hidden dims in a current layer to the next MLP layer'
                            })
  combine_feat_method: str = field(default='individual_mlps_on_cat_and_numerical_feats_then_concat',
                                    metadata={
                                        'help': 'method to combine categorical and numerical features, '
                                                'see README for all the method'
                                    })
  mlp_dropout: float = field(default=0.1,
                              metadata={
                                'help': 'dropout ratio used for MLP layers'
                              })
  numerical_bn: bool = field(default=True,
                              metadata={
                                  'help': 'whether to use batchnorm on numerical features'
                              })
  use_simple_classifier: str = field(default=True,
                                      metadata={
                                          'help': 'whether to use single layer or MLP as final classifier'
                                      })
  mlp_act: str = field(default='relu',
                        metadata={
                            'help': 'the activation function to use for finetuning layers',
                            'choices': ['relu', 'prelu', 'sigmoid', 'tanh', 'linear']
                        })
  gating_beta: float = field(default=0.2,
                              metadata={
                                  'help': "the beta hyperparameters used for gating tabular data "
                                          "see https://www.aclweb.org/anthology/2020.acl-main.214.pdf"
                              })

  def __post_init__(self):
      assert self.column_info != self.column_info_path
      if self.column_info is None and self.column_info_path:
          with open(self.column_info_path, 'r') as f:
              self.column_info = json.load(f)

In [22]:
text_cols = ['HateSpeech','Target']
cat_cols = ['Target']
# cat_cols = ['Clothing ID', 'Division Name', 'Department Name', 'Class Name']
numerical_cols = ['Length']

column_info_dict = {
    'text_cols': text_cols,
    'cat_cols' : cat_cols,
    'num_cols' : numerical_cols,
    'label_col': 'BestIntent',
    'label_list': [0.0,1.0,2.0,3.0]
}

# ['Informative', 'Questioning','Positive','Denouncing']


model_args = ModelArguments(
    model_name_or_path='bert-base-uncased'
)

data_args = MultimodalDataTrainingArguments(
    data_path='.',
    column_info=column_info_dict,
    task='classification'
)

training_args = TrainingArguments(
    output_dir="model_name",
    logging_dir="runs",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    per_device_train_batch_size=32,
    num_train_epochs=10,
    evaluation_strategy = "steps",
    logging_steps=25,
)

set_seed(training_args.seed)

In [23]:
tokenizer_path_or_name = model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path
print('Specified tokenizer: ', tokenizer_path_or_name)
tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_path_or_name,
    cache_dir=model_args.cache_dir,
)

Specified tokenizer:  bert-base-uncased


In [24]:
data_df.columns

Index(['HateSpeech', 'BestIntent', 'Target', 'Length'], dtype='object')

In [25]:
# Get Datasets
train_dataset, val_dataset, test_dataset = load_data_from_folder(
    folder_path = data_args.data_path,
    text_cols = data_args.column_info['text_cols'],
    tokenizer = tokenizer,
    label_col=data_args.column_info['label_col'],
    label_list=data_args.column_info['label_list'],
    categorical_cols=data_args.column_info['cat_cols'],
    numerical_cols=data_args.column_info['num_cols'],
    # categorical_encode_type=None,
    sep_text_token_str=tokenizer.sep_token,
)

INFO:multimodal_transformers.data.data_utils:1 numerical columns
INFO:multimodal_transformers.data.data_utils:8 categorical columns
INFO:multimodal_transformers.data.data_utils:1 numerical columns
INFO:multimodal_transformers.data.load_data:Text columns: ['HateSpeech', 'Target']
INFO:multimodal_transformers.data.load_data:Raw text example: Women must not have equal rights, they are merely a burden to society, since they want to steal men's rights. [SEP] 2
INFO:multimodal_transformers.data.data_utils:8 categorical columns
INFO:multimodal_transformers.data.data_utils:1 numerical columns
INFO:multimodal_transformers.data.load_data:Text columns: ['HateSpeech', 'Target']
INFO:multimodal_transformers.data.load_data:Raw text example: The feminists have won. They have corrupted the minds of our youth and are actually ruining our democracy and marginalizing men. [SEP] 2
INFO:multimodal_transformers.data.data_utils:8 categorical columns
INFO:multimodal_transformers.data.data_utils:1 numerical co

In [26]:
num_labels = len(np.unique(train_dataset.labels))
num_labels

4

In [27]:
config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
tabular_config = TabularConfig(num_labels=num_labels,
                               cat_feat_dim = train_dataset.cat_feats.shape[1],
                               numerical_feat_dim=train_dataset.numerical_feats.shape[1],
                               **vars(data_args))
config.tabular_config = tabular_config

In [28]:
model = AutoModelWithTabular.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        config=config,
        cache_dir=model_args.cache_dir
    )

/home/ayushmt119/.local/lib/python3.10/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertWithTabular: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertWithTabular from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertWithTabular from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassi

In [29]:
import numpy as np
from scipy.special import softmax
from sklearn.metrics import (
    auc,
    precision_recall_curve,
    roc_auc_score,
    f1_score,
    confusion_matrix,
    matthews_corrcoef,
)

def calc_classification_metrics(p: EvalPrediction):
  # print(p.predictions[0])
  # print(len(p.predictions[0]))
  # print('-0-------------------')
  # print(p.predictions[1], type(p.predictions[0]), type(p.predictions[1]), len(p.predictions), type(p.predictions))

  # for i in range(len(p.predictions)):
  #   p.predictions[i] = torch.from_numpy(np_array)
  # pred2 = torch.tensor(p.predictions)
  # pred2 = [torch.from_numpy(item).float() for item in p.predictions]
  pred_labels = np.argmax(p.predictions[0], axis=1)
  # print(pred_labels)
  pred_scores = softmax(p.predictions[0], axis=1)
  print(pred_scores)
  labels = p.label_ids
  labels = [int(x) for x in labels]
  # print('p.label_ids \n', p.label_ids)
  # if len(np.unique(labels)) == 5: 
  #     # roc_auc_pred_score = roc_auc_score(labels, pred_scores, multi_class='ovo')
  #     precisions, recalls, thresholds = precision_recall_curve(labels,
  #                                                               pred_scores)
  #     fscore = (2 * precisions * recalls) / (precisions + recalls)
  #     fscore[np.isnan(fscore)] = 0
  #     ix = np.argmax(fscore)
  #     threshold = thresholds[ix].item()
  #     # pr_auc = auc(recalls, precisions)
  #     tn, fp, fn, tp = confusion_matrix(labels, pred_labels, labels=[0.0, 1.0, 2.0, 3.0, 4.0]).ravel()
  #     result = {
  #         'threshold': threshold,
  #               'recall': recalls[ix].item(),
  #               'precision': precisions[ix].item(), 'f1': fscore[ix].item(),
  #               'tn': tn.item(), 'fp': fp.item(), 'fn': fn.item(), 'tp': tp.item()
  #               }
  # else:
  acc = (pred_labels == labels).mean()
  f1 = f1_score(y_true=labels, y_pred=pred_labels, average="macro")
  result = {
      "acc": acc,
      "f1": f1,
      "avg": (acc + f1) / 2,
      "mcc": matthews_corrcoef(labels, pred_labels)
  }

  return result

In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=calc_classification_metrics,
)

In [31]:
%%time
trainer.train()

/home/ayushmt119/.local/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: japneet50. Use `wandb login --relogin` to force relogin


RuntimeError: Caught RuntimeError in replica 1 on device 1.
Original Traceback (most recent call last):
  File "/home/ayushmt119/.local/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
  File "/home/ayushmt119/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/ayushmt119/.local/lib/python3.10/site-packages/multimodal_transformers/model/tabular_transformers.py", line 114, in forward
    combined_feats = self.tabular_combiner(pooled_output,
  File "/home/ayushmt119/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/ayushmt119/.local/lib/python3.10/site-packages/multimodal_transformers/model/tabular_combiner.py", line 376, in forward
    numerical_feats = self.num_mlp(numerical_feats)
  File "/home/ayushmt119/.local/lib/python3.1

rank 6
```
Step	Training Loss	Validation Loss	Acc	F1	Avg	Mcc
25	1.146600	1.092061	0.612121	0.189850	0.400985	0.000000
50	1.075100	1.088367	0.612121	0.189850	0.400985	0.000000
75	1.141400	1.095874	0.612121	0.189850	0.400985	0.000000
100	1.158700	1.085643	0.612121	0.189850	0.400985	0.000000
125	1.099100	1.076692	0.612121	0.189850	0.400985	0.000000
150	1.084500	1.106865	0.612121	0.189850	0.400985	0.000000
175	1.145500	1.093524	0.612121	0.189850	0.400985	0.000000
200	1.059400	1.124229	0.612121	0.189850	0.400985	0.000000
225	1.070700	1.112222	0.612121	0.189850	0.400985	0.000000
250	1.089300	1.120609	0.587879	0.195381	0.391630	-0.035419
275	0.923800	1.279086	0.509091	0.223706	0.366399	-0.010010
300	0.951100	1.261840	0.460606	0.236134	0.348370	0.011618
325	0.869800	1.265962	0.500000	0.254206	0.377103	0.024408
350	0.747100	1.458164	0.512121	0.235004	0.373563	0.000465
```

rank 4 
```
Step	Training Loss	Validation Loss	Acc	F1	Avg	Mcc
25	0.966600	0.859716	0.732087	0.211331	0.471709	0.000000
50	0.882500	0.843997	0.732087	0.211331	0.471709	0.000000
75	0.862600	0.830599	0.732087	0.211331	0.471709	0.000000
100	0.900500	0.854101	0.732087	0.211331	0.471709	0.000000
125	0.858300	0.868348	0.722741	0.218710	0.470726	-0.003199
150	0.885100	0.849566	0.732087	0.211331	0.471709	0.000000
175	0.835100	0.878387	0.719626	0.218019	0.468823	-0.013392
200	0.708600	0.885336	0.725857	0.242879	0.484368	0.079818
225	0.792600	0.915518	0.707165	0.251058	0.479112	0.033643
250	0.750000	0.953677	0.669782	0.265696	0.467739	0.053696
275	0.604400	1.129580	0.619938	0.292408	0.456173	0.076044
300	0.572300	1.112399	0.666667	0.255551	0.461109	0.049204
325	0.575600	1.052724	0.610592	0.288537	0.449564	0.033591
350	0.418600	1.215113	0.641745	0.326217	0.483981	0.093172
375	0.403700	1.326293	0.651090	0.257527	0.454309	0.023636
```
rank 3
```
avg peak at 66/29

Step	Training Loss	Validation Loss	Acc	F1	Avg	Mcc
25	0.967000	0.856174	0.732087	0.211331	0.471709	0.000000
50	0.879400	0.844840	0.732087	0.211331	0.471709	0.000000
75	0.862300	0.832946	0.732087	0.211331	0.471709	0.000000
100	0.908000	0.845187	0.732087	0.211331	0.471709	0.000000
125	0.851300	0.837765	0.732087	0.211331	0.471709	0.000000
150	0.884000	0.834059	0.732087	0.211331	0.471709	0.000000
175	0.829400	0.873287	0.732087	0.211331	0.471709	0.000000
200	0.709700	0.954770	0.725857	0.227926	0.476892	0.046604
225	0.815700	0.910960	0.676012	0.215804	0.445908	-0.066464
250	0.747000	0.975431	0.679128	0.284749	0.481938	0.060783
275	0.615600	1.179581	0.576324	0.280858	0.428591	0.046069
300	0.552100	1.178694	0.626168	0.287040	0.456604	0.012996
325	0.568800	1.130639	0.598131	0.271206	0.434668	0.002694
350	0.393000	1.428724	0.566978	0.286290	0.426634	0.033320
375	0.370900	1.375929	0.654206	0.275404	0.464805	0.033782
400	0.401700	1.252681	0.666667	0.296136	0.481402	0.092566
425	0.268900	1.513582	0.591900	0.303933	0.447917	0.058049
450	0.257200	1.729297	0.545171	0.296681	0.420926	0.058043
475	0.256100	1.639327	0.607477	0.273330	0.440403	0.026243
```

rank2 

```
Step	Training Loss	Validation Loss	Acc	F1	Avg	Mcc
25	0.966900	0.857973	0.732087	0.211331	0.471709	0.000000
50	0.881000	0.841156	0.732087	0.211331	0.471709	0.000000
75	0.863400	0.827716	0.732087	0.211331	0.471709	0.000000
100	0.899900	0.852294	0.732087	0.211331	0.471709	0.000000
125	0.870400	0.850865	0.732087	0.211331	0.471709	0.000000
150	0.877000	0.860997	0.732087	0.211331	0.471709	0.000000
175	0.862700	0.851289	0.728972	0.210811	0.469891	-0.025438
200	0.729100	0.863609	0.728972	0.220310	0.474641	0.039371
225	0.823500	0.891966	0.700935	0.231320	0.466127	0.016192
250	0.759000	0.948081	0.682243	0.252255	0.467249	0.071435
275	0.617200	1.209996	0.554517	0.262806	0.408661	0.047829
300	0.598400	1.041994	0.691589	0.258023	0.474806	0.074478
325	0.589200	1.119987	0.619938	0.298537	0.459237	0.068191
350	0.414600	1.310834	0.595016	0.290354	0.442685	0.083669
375	0.432200	1.276930	0.638629	0.261000	0.449815	0.025174
400	0.434900	1.273736	0.644860	0.304040	0.474450	0.101716
425	0.273900	1.481061	0.585670	0.279280	0.432475	0.052017
450	0.277100	1.671477	0.566978	0.270263	0.418621	0.042627
475	0.279200	1.669125	0.595016	0.299922	0.447469	0.076451
500	0.258800	1.496269	0.660436	0.269614	0.465025	0.021298
```

added length, rank1

```
fixed-
 [520/830 05:22 < 03:13, 1.60 it/s, Epoch 6.25/10]
Step	Training Loss	Validation Loss	Acc	F1	Avg	Mcc
25	1.122000	1.069787	0.624242	0.192164	0.408203	0.000000
50	1.067800	1.081804	0.624242	0.192164	0.408203	0.000000
75	1.111700	1.069398	0.624242	0.192164	0.408203	0.000000
100	1.125600	1.071947	0.624242	0.192164	0.408203	0.000000
125	1.074300	1.069981	0.624242	0.192164	0.408203	0.000000
150	1.063000	1.083102	0.624242	0.192164	0.408203	0.000000
175	1.116600	1.091780	0.624242	0.192164	0.408203	0.000000
200	1.022100	1.122842	0.624242	0.192523	0.408383	0.026019
225	1.012200	1.102715	0.624242	0.203575	0.413909	0.036852
250	1.027800	1.107622	0.593939	0.204934	0.399437	-0.000797
275	0.761700	1.362213	0.466667	0.250605	0.358636	0.020176
300	0.867000	1.273436	0.593939	0.245871	0.419905	0.067190
325	0.695500	1.425792	0.454545	0.251646	0.353096	0.021140
350	0.538200	1.539966	0.554545	0.260785	0.407665	0.044653
375	0.418900	1.659600	0.521212	0.259665	0.390438	0.049253
400	0.456100	1.638591	0.466667	0.252651	0.359659	0.016008
425	0.379300	1.692099	0.524242	0.253311	0.388776	0.034999
450	0.267700	2.008947	0.460606	0.245992	0.353299	0.005471
475	0.223900	1.944380	0.557576	0.275858	0.416717	0.070413
500	0.239600	2.059766	0.493939	0.255033	0.374486	0.030860
```
```
Step	Training Loss	Validation Loss	Acc	F1	Acc And F1	Mcc
25	1.128300	1.148606	0.578788	0.183301	0.381045	0.000000
50	1.130200	1.154853	0.578788	0.183301	0.381045	0.000000
75	1.064300	1.156120	0.578788	0.183301	0.381045	0.000000
100	1.093600	1.157456	0.578788	0.183301	0.381045	0.000000
125	1.093400	1.159407	0.578788	0.183301	0.381045	0.000000
150	1.100900	1.163721	0.578788	0.183301	0.381045	0.000000
175	1.070900	1.176999	0.578788	0.183301	0.381045	0.000000
200	1.013800	1.232586	0.566667	0.181202	0.373934	-0.045518
225	1.000900	1.222369	0.545455	0.197098	0.371276	-0.046380
250	1.056300	1.206484	0.551515	0.199405	0.375460	-0.016469
275	0.894800	1.288560	0.518182	0.236412	0.377297	0.005804
300	0.816400	1.380967	0.454545	0.243585	0.349065	-0.018921
325	0.835400	1.383750	0.509091	0.242468	0.375780	-0.001184
350	0.706800	1.470209	0.475758	0.241841	0.358799	-0.017662
375	0.673900	1.520240	0.454545	0.236691	0.345618	-0.019256
400	0.686000	1.521897	0.436364	0.236129	0.336246	-0.023581
```

made target categorical
```
Step	Training Loss	Validation Loss	Acc	F1	Acc And F1	Mcc
25	1.157000	1.116611	0.587879	0.185115	0.386497	0.000000
50	1.115700	1.109924	0.587879	0.185115	0.386497	0.000000
75	1.080000	1.116495	0.587879	0.185115	0.386497	0.000000
100	1.082300	1.115990	0.587879	0.185115	0.386497	0.000000
125	1.080800	1.117029	0.587879	0.185115	0.386497	0.000000
150	1.063300	1.123146	0.587879	0.185115	0.386497	0.000000
175	1.130000	1.109484	0.587879	0.185115	0.386497	0.000000
200	1.049400	1.129704	0.587879	0.185468	0.386674	0.023209
225	1.051100	1.171464	0.506061	0.209096	0.357578	-0.033015
250	1.051500	1.140421	0.587879	0.185468	0.386674	0.023209
275	0.932800	1.210289	0.563636	0.181287	0.372461	-0.046280
300	0.919800	1.230302	0.527273	0.196865	0.362069	-0.056997
325	0.936900	1.208007	0.533333	0.241502	0.387418	0.007843
350	0.849200	1.221206	0.542424	0.230148	0.386286	-0.002701
375	0.775700	1.266828	0.521212	0.240299	0.380756	0.006790
400	0.757800	1.276696	0.527273	0.243936	0.385604	0.017956
```

scale 3 with oversampling 
```
Step	Training Loss	Validation Loss	Acc	F1	Acc And F1	Mcc
25	1.100600	1.193198	0.554545	0.178363	0.366454	0.000000
50	1.097500	1.174873	0.554545	0.178363	0.366454	0.000000
75	1.102400	1.160298	0.554545	0.178363	0.366454	0.000000
100	1.078600	1.162747	0.554545	0.178363	0.366454	0.000000
125	1.076600	1.153444	0.554545	0.178363	0.366454	0.000000
150	1.075200	1.182141	0.554545	0.178363	0.366454	0.000000
175	1.103400	1.153877	0.554545	0.178363	0.366454	0.000000
200	1.033700	1.175852	0.551515	0.178082	0.364799	-0.002979
225	0.963500	1.180197	0.551515	0.188438	0.369977	0.017354
250	1.052500	1.167078	0.545455	0.211266	0.378360	0.035719
275	0.893200	1.266699	0.512121	0.228253	0.370187	0.017802
300	0.903800	1.262344	0.484848	0.283844	0.384346	0.045828
325	0.912300	1.252127	0.506061	0.256502	0.381281	0.050560
350	0.779200	1.281733	0.490909	0.258511	0.374710	0.041559
375	0.721400	1.292633	0.503030	0.275099	0.389065	0.055470
400	0.712700	1.336567	0.500000	0.257371	0.378685	0.030039
```

scale 3
```
Step	Training Loss	Validation Loss	Acc	F1	Acc And F1	Mcc
25	1.093200	1.148656	0.569697	0.181467	0.375582	0.000000
50	1.145400	1.128789	0.569697	0.181467	0.375582	0.000000
75	1.110000	1.132535	0.569697	0.181467	0.375582	0.000000
100	1.078800	1.133871	0.569697	0.181467	0.375582	0.000000
125	1.070100	1.145953	0.569697	0.181467	0.375582	0.000000
150	1.099000	1.133926	0.569697	0.181467	0.375582	0.000000
175	1.114000	1.179797	0.569697	0.181467	0.375582	0.000000
200	1.023100	1.135850	0.569697	0.181467	0.375582	0.000000
225	1.011700	1.169308	0.572727	0.191818	0.382273	0.071090
250	1.054100	1.178249	0.572727	0.191818	0.382273	0.071090
275	0.969100	1.236769	0.484848	0.254498	0.369673	0.026251
300	0.966000	1.258493	0.548485	0.233884	0.391185	0.029828
325	0.925800	1.238145	0.548485	0.219284	0.383885	0.016318
350	0.827000	1.274115	0.527273	0.246553	0.386913	0.042564
375	0.818100	1.302960	0.490909	0.252829	0.371869	0.014737
400	0.783300	1.332106	0.478788	0.216915	0.347852	-0.035805
```

Results using bert-base-uncased, scale5
```
Step	Training Loss	Validation Loss	Acc	F1	Acc And F1	Mcc
25	1.171600	1.149500	0.575758	0.182692	0.379225	0.000000
50	1.139400	1.144753	0.575758	0.182692	0.379225	0.000000
75	1.158300	1.141287	0.575758	0.182692	0.379225	0.000000
100	1.159100	1.133854	0.575758	0.182692	0.379225	0.000000
125	1.124000	1.158157	0.575758	0.182692	0.379225	0.000000
150	1.131500	1.151931	0.575758	0.182692	0.379225	0.000000
175	1.104900	1.165865	0.575758	0.182692	0.379225	0.000000
200	1.108700	1.176736	0.575758	0.206082	0.390920	0.068544
225	1.073400	1.184515	0.566667	0.190289	0.378478	0.017425
250	1.069300	1.184888	0.557576	0.207061	0.382318	0.036214
275	0.991300	1.268941	0.542424	0.199862	0.371143	0.015351
300	0.954100	1.300586	0.487879	0.221395	0.354637	0.001057
325	0.883400	1.333021	0.487879	0.210279	0.349079	-0.014522
350	0.816300	1.381134	0.530303	0.215084	0.372694	0.011525
375	0.785700	1.405480	0.521212	0.220570	0.370891	0.020994
400	0.812400	1.422695	0.463636	0.230714	0.347175	-0.005237
```

In [ ]:
metrics = trainer.evaluate(eval_dataset=test_dataset)

In [ ]:
metrics

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
test_dataset.labels

In [ ]:
preds = trainer.predict(test_dataset)

In [ ]:
# for x in preds[0][0]:
#   print(x)
#   print(len(x))

In [ ]:
pred_labels = np.argmax(preds[0][0], axis=1)

In [ ]:
pred_labels

In [ ]:
preds

In [ ]:
from sklearn.metrics import confusion_matrix
# true, pred
confusion_matrix(preds[1], pred_labels)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(preds[1], pred_labels, labels=[0,1,2,3])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1,2,3])
disp.plot()
# plt.show()

In [ ]:
%tensorboard --logdir ./logs/runs --port=6006